<a href="https://colab.research.google.com/github/a00700c/AI-teamproject/blob/main/concrete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("/content/sample_data/Concrete_Data_Yeh.csv")

In [10]:
df.head()
#CRIM 근방 범죄율
#ZN 주택지 비율
#INDUS 상업적 비지니스에 활용되지 않는 농지 면적
#CHAS 경계선에 강에 있는지 여부
#NOX 산화 질소 농도
#RM 자택당 평균 방 갯수
#AGE 1940 년 이전에 건설된 비율
#DIS 5 개의 보스턴 고용 센터와의 거리에 다른 가주이 부여
#RAD radial 고속도로와의 접근성 지수
#TAX 10000달러당 재산세
#PTRATIO 지역별 학생-교사 비율
#B 지역의 흑인 지수
#LSTAT 빈공층의 비율

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [11]:
df.shape

(1030, 9)

In [12]:
target = df['csMPa']
data = df.drop(['csMPa'], axis = 1)

In [13]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: csMPa, dtype: float64

In [14]:
data.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [15]:
#XBGRegressor 회귀 모델 사용하기 위한 import
from xgboost import XGBRegressor 

In [16]:
# 데이터 plot 과 어떤 feature가 가중치가 높은지 확인하기 위한 라이브러리 import
from xgboost import plot_importance, plot_tree
import graphviz
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
# 모델의 하이퍼 파라미터 설정
# 의사결정 트리수 (n_estimators) 100개로 정의
# 사용할 코어수 (n_jobs) 최대치로 사용
# learing_rate 초기값 0.01 사용
# 모델 여러개 돌리기 위해 사이킷런의 그리드 서치 사용
from sklearn.model_selection import GridSearchCV, train_test_split
xgb = XGBRegressor()

In [18]:
#하이퍼 파라미터 종류
# learning_rate // 학습률
# n_estimators // 학습기의 갯수(반복 수행 횟수)
# min_child_weight leaf와 유사, 과적합 조절용
# max_depth 트리의 최대 깊이
# subsample 샘플링하는 비율
# early_stopping_rounds 더 이상 비용 평가 지표가 감소하지 않는 최대 반복 횟수
# eval_metric 반복 수행 시 사용하는 비용 평가 지표
# eval_set 평가를 수행하는 별도의 검증 데이터 세트, 일반적으로 검증 세트에서 반복적으로 비용 감소 성능 평가
params = {'n_estimators': [i * 50 for i in range(10,20)], 
          'learning_rate': [i * 0.01 for i in range(0, 20)], 
          'n_jobs': [-1], 
          'max_depth': [i * 4 for i in range(5)]}

In [19]:
x_train, t_test, x_valid, t_valid = train_test_split(data, target, test_size=0.2, shuffle=True, random_state=7)

In [20]:
x_train, x_valid

(     cement   slag  flyash  water  superplasticizer  coarseaggregate  \
 15    380.0    0.0     0.0  228.0               0.0            932.0   
 308   277.1    0.0    97.4  160.6              11.8            973.9   
 159   389.9  189.0     0.0  145.9              22.0            944.7   
 482   446.0   24.0    79.0  162.0              11.6            967.0   
 417   194.7    0.0   100.5  170.2               7.5            998.0   
 ..      ...    ...     ...    ...               ...              ...   
 919   313.0    0.0     0.0  178.0               8.0           1000.0   
 211   230.0    0.0   118.3  195.5               4.6           1029.4   
 502   491.0   26.0   123.0  210.0               3.9            882.0   
 537   480.0    0.0     0.0  192.0               0.0            936.2   
 175   362.6  189.0     0.0  164.9              11.6            944.7   
 
      fineaggregate  age  
 15           670.0   90  
 308          875.6  100  
 159          755.8   56  
 482          

In [21]:
gs = GridSearchCV(xgb, params, scoring='neg_mean_absolute_percentage_error', cv=5)

In [22]:
gs.fit(x_train, x_valid)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[18:40:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:lin

GridSearchCV(cv=5, estimator=XGBRegressor(),
             param_grid={'learning_rate': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05,
                                           0.06, 0.07, 0.08, 0.09, 0.1, 0.11,
                                           0.12, 0.13, 0.14, 0.15, 0.16, 0.17,
                                           0.18, 0.19],
                         'max_depth': [0, 4, 8, 12, 16],
                         'n_estimators': [500, 550, 600, 650, 700, 750, 800,
                                          850, 900, 950],
                         'n_jobs': [-1]},
             scoring='neg_mean_absolute_percentage_error')

In [23]:
cv_result_df = pd.DataFrame(gs.cv_results_)
cv_result_df.sort_values(by=['rank_test_score'], inplace=True)

In [24]:
cv_result_df[['params', 'mean_test_score', 'rank_test_score']].head(100)

,params,mean_test_score,rank_test_score
519,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",-0.105227,1
518,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",-0.105285,2
517,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",-0.105419,3
516,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",-0.105529,4
618,"{'learning_rate': 0.12, 'max_depth': 4, 'n_est...",-0.105589,5
...,...,...,...
179,"{'learning_rate': 0.03, 'max_depth': 8, 'n_est...",-0.107787,96
371,"{'learning_rate': 0.07, 'max_depth': 8, 'n_est...",-0.107809,97
719,"{'learning_rate': 0.14, 'max_depth': 4, 'n_est...",-0.107813,98
178,"{'learning_rate': 0.03, 'max_depth': 8, 'n_est...",-0.107819,99


In [25]:
best = gs.best_estimator_

In [26]:
pred = best.predict(t_test)

In [27]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error
print('RMSE : ', mean_squared_error(pred, t_valid))
print('MAPE : ', mean_absolute_percentage_error(pred, t_valid))

RMSE :  10.425992047690958
MAPE :  0.07328646553548565


In [28]:
pred

array([12.26123  , 14.211094 , 34.095783 , 10.210182 , 54.708317 ,
       28.425873 , 62.875523 , 10.501048 , 42.82818  , 13.465069 ,
       17.307068 , 50.61765  , 15.646004 , 54.00814  , 14.610616 ,
       34.95513  , 69.89719  , 43.391087 , 45.662365 , 19.725662 ,
       36.06326  , 64.10797  , 34.811764 , 46.878933 , 79.49859  ,
       18.098852 , 39.547134 ,  6.2369657, 20.08098  , 42.840405 ,
       25.130112 , 36.105778 , 45.96496  , 41.02597  , 29.373526 ,
       40.202286 , 81.364746 , 49.95542  , 55.120655 , 60.342075 ,
       61.842983 , 27.87019  , 22.147774 , 21.713346 , 48.950806 ,
       53.16912  , 37.46289  , 42.39225  , 20.83963  , 55.041664 ,
       30.03327  , 42.2175   , 25.042053 , 21.944124 , 42.52312  ,
       36.486374 , 15.471498 , 56.155224 , 39.774357 ,  8.624939 ,
       41.515385 , 44.15601  , 64.110725 , 20.967474 , 33.26688  ,
       33.29895  , 31.034916 , 38.58504  , 38.473198 , 24.346687 ,
       13.931602 , 33.19496  , 17.629595 , 43.14277  ,  4.7053

In [29]:
result = pd.DataFrame(pred)

In [30]:
result.to_excel('결과.xlsx')
t_valid.to_excel('테스트대상.xlsx')

In [31]:
print('정확도', r2_score(pred, t_valid))

정확도 0.9602235543398584
